In [1]:
import pandas as pd
import os, glob

from owlready2 import *
import owlready2
print(owlready2.VERSION)


import importlib.util
import sys
spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

%load_ext autoreload
%autoreload 2


0.40


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
from dotenv import load_dotenv
load_dotenv('.env')
import json, requests
import importlib.util
import sys

spec = importlib.util.spec_from_file_location("llm", "../../../utils/llm.py")
h = importlib.util.module_from_spec(spec)
sys.modules["llm"] = h
spec.loader.exec_module(h)

URL = os.getenv("KG_URL_FCT")
TOK3N = os.getenv("KG_TOKEN")

In [3]:
onto = get_ontology("WIP.owl").load()
dIDct = u.createDict(onto) 

PBNThing
BenefReturn
Benef
Article
Risk
ISO_Scale
RiskHealth
RiskType
Stakeholder
Stakeholder_Type
Technology
ISO_Impact
ISO_Purpose
StakeholderGroup
StakeholderSubgroup
TechGroup
TechSubGroup
Mitigation
BP_Enabler
BP_Transmission
Blueprint
BP_Scale
BP_Phase
BP_Permanent
BP_Type
BP_Intervention
PBNCategory
RiskGroup
RiskSubgroup
CAO_Group
CAO_Subgroup
BP_Theme
BP_Category
aBlueprint


In [7]:
comments = u.checkComments(onto)

ID: 0 	 Author: Luc Jonveaux
ID: 1 	 License: CC BY-NC-SA
ID: 2 	 Language: English
ID: 3 	 Project: PROBONO
ID: 4 	 Task: T3.5
ID: 5 	 Description: Creation of a knowledge graph based on a litterature review, augmented by use of LLMs.
ID: 6 	 Changes from: 0.0
ID: 7 	 Changes: Creation of the knowledge graph
ID: 8 	 Changes from: 0.1
ID: 9 	 Changes: Adding relations, simplified ontology, closest items, and descriptions
ID: 10 	 Creation: 11/11/2023


# Testing custom SPARQL queries

In [7]:
from IPython.display import display, Markdown

[autoreload of rdfutils failed: Traceback (most recent call last):
  File "/home/kelu/projets/T3.5/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/kelu/projets/T3.5/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 168, in reload
    raise ModuleNotFoundError(f"spec not found for the module {name!r}", name=name)
ModuleNotFoundError: spec not found for the module 'rdfutils'
]


In [8]:
def EX(REQ):
    return list(default_world.sparql(REQ))
    
def ask(QUESTION,prefix="sparql",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed=""):
    CONTEXT  = "You are an expert in the sparql language."
    CONTEXT += "You will have to provide  a sparql request that counts the number of classes in a knowledge graph, \
          please answer with the request as between ```sparql ``` tags, and then provide details of how the request is built."
    REQ = {
        "context": CONTEXT,
        "question": QUESTION,
        "model": MODEL,
        "token": TOK3N,
        "overwrite": overwrite,
        "source": "local-sparqlqueries",
        "seed" : seed
    }

    H = h.hashme(CONTEXT+QUESTION+seed)
    cached = "cache/"+prefix+"_"+H+".json"
    if not os.path.isfile(cached) or overwrite:
        x = requests.post(URL+"ask/", json = REQ)
        #print(x.text)
        answer = json.loads(x.text)["answer"]
        h.svt(cached,answer)
    else:
        answer = h.ldt(cached)
    A = [x.replace("sparql\n","").strip() for x in answer.split("```") if len(x)]

    display(Markdown((A[-1])))
    display(Markdown("--------"))
    display(Markdown("```sparql\n"+(A[-2])+"\n```"))
    display(Markdown("--------"))
    B = EX(A[-2])
    return B

In [9]:
specifics ="""The ontology we are reviewing has several classes:
* 'Risk': A specific risk
* 'Mitigation': A mitigation against a risk
* 'Technology': A technology used to mitigate a risk
* 'T' : A category used to classify Technology
* 'Stakeholder': People related to risks
* 'Article': Sources of information
They are linked using the following properties:
* A 'Risk' has a 'Mitigation' : noted with the property : 'has_RiskMitigation' 
* A 'Risk' has a 'Technology' : noted with the property : 'has_RiskTechnology'
* A 'Risk' impacts a 'Stakeholder' : noted with the property : Risk->'has_RiskSubject'->Stakeholder . Inverse is 'has_SubjectRisk' .
* A 'Risk' can be mitigated by a 'Stakeholder' :  noted with the property : 'has_RiskOwner'
* A 'T' contains 'Technology', noted with the property:  Technology->'has_T'->T . Inverse is 'has_GroupTech' .
Note that all classes have capital letters as part of their names.

This ontology can be added in the sparql request introducing:
PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>
"""

In [10]:
A = ask(specifics+"How do I count the number of classes",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed="")
A

To count the number of classes in the knowledge graph, the SPARQL query uses a SELECT clause with the COUNT aggregate function to count the instances of a class. The WHERE clause specifies that the ?class variable must be an instance of the owl:Class. By using the PREFIX directive, the query associates the pbn namespace with the base ontology URI, enabling the use of the ontology's classes.

The query does not include specific class names as it aims to retrieve a general count of all classes in the ontology. This way, it can capture any additional classes that might be added to the ontology in the future.

--------

```sparql
PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT (COUNT(?class) AS ?classCount)
WHERE {
  ?class a owl:Class
}
```

--------

[[34]]

In [12]:
A = ask(specifics+"How do I count how many Technology belong to each T, and sort in ascending order (largest T first), keeping the top 7?",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed="3") # gpt-4-0613
A

This SPARQL query accomplishes the task of counting how many technologies belong to each TechGroup and sorting them in ascending order (largest TechGroup first) while keeping the top 7. Here's a breakdown of the query:

1. The SPARQL query starts with a PREFIX declaration to define the namespace prefix for the ontology: `PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>`. This prefix will be used to simplify the notation of the ontology classes and properties in the query.

2. The SELECT clause includes the variables `?techGroup` and `(COUNT(?technology) AS ?techCount)`. This part of the query specifies the variables to be selected in the results. `?techGroup` will represent the label of the TechGroup, and `?techCount` will represent the count of technologies associated with each TechGroup.

3. The WHERE clause includes the triple patterns to match the relationships between technologies and TechGroups. It specifies that a `?technology` entity is of type `pbn:Technology` and has the property `pbn:has_TechGroup` with the object `?techGroupObj`. Additionally, it retrieves the label of the `?techGroupObj` using the `rdfs:label` property and assigns it to the variable `?techGroup`.

4. The GROUP BY clause groups the results based on the variable `?techGroup`, which represents the label of the TechGroup.

5. The ORDER BY clause sorts the results in descending order of `?techCount`, which represents the count of technologies for each TechGroup.

6. The LIMIT 7 clause ensures that only the top 7 results are returned, representing the largest TechGroups based on the count of technologies.

Overall, this SPARQL query effectively counts the number of technologies belonging to each TechGroup and provides the results in the desired order and limit.

--------

```sparql
PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>
SELECT ?techGroup (COUNT(?technology) AS ?techCount)
WHERE {
  ?technology a pbn:Technology ;
             pbn:has_TechGroup ?techGroupObj .
  ?techGroupObj a pbn:TechGroup ;
               rdfs:label ?techGroup .
} 
GROUP BY ?techGroup
ORDER BY DESC(?techCount)
LIMIT 7
```

--------

[['Healthcare', 5802],
 ['Data analytics', 3322],
 ['Urban planning', 2131],
 ['Infrastructure management', 2017],
 ['Communication', 1813],
 ['Digital and information management', 1684],
 ['Sensors and monitoring', 1270]]

In [13]:
EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>
SELECT ?T (COUNT(?technology) AS ?techCount)
WHERE {
  ?technology a pbn:Technology ;
             pbn:has_T ?TObj .
  ?TObj a pbn:T ;
               rdfs:label ?T .
} 
GROUP BY ?T
ORDER BY DESC(?techCount)
LIMIT 7""")

[['Healthcare', 5802],
 ['Data analytics', 3322],
 ['Urban planning', 2131],
 ['Infrastructure management', 2017],
 ['Communication', 1813],
 ['Digital and information management', 1684],
 ['Sensors and monitoring', 1270]]

In [9]:
A = ask(specifics+"How do I identify the T, and its label, that has the most Technology items linked to a Risk?",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed="5")
A

This SPARQL query is identifying the 'TechGroup' and its label that has the most 'Technology' items linked to a 'Risk'. The query consists of the following elements:

1. First, we specify the prefix for our ontology using `PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>`.

2. We then use a SELECT query to retrieve the 'TechGroup' and count the number of 'Technology' items associated with a 'Risk'. We alias the count as `?numTechnologies`.

3. In the WHERE clause, we specify the pattern where a 'Risk' is linked to a 'Technology' through the property `pbn:has_RiskTechnology`, and the 'Technology' is linked to a 'TechGroup' using the property `pbn:has_TechGroup`.

4. We use the GROUP BY clause to group the results by the 'TechGroup'.

5. We then use the ORDER BY clause to sort the results in descending order based on the count of 'Technology' items.

6. Finally, we use the LIMIT 1 clause to retrieve only the top 'TechGroup' with the most linked 'Technology' items.

--------

```sparql
PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT ?techGroup (COUNT(?technology) AS ?numTechnologies)
WHERE {
  ?risk a pbn:Risk ;
        pbn:has_RiskTechnology ?technology .
  ?technology pbn:has_TechGroup ?techGroup .
}
GROUP BY ?techGroup
ORDER BY DESC(?numTechnologies)
LIMIT 1```

--------

[[WIP.PBN__TechGroup_2, 12729]]

In [10]:
A = ask(specifics+"Let's take the 'pbn:PBN__Risk_1', one of the Risk instance. How do I identify the different individual T to which the Technology instances linked to the Risk (through has_RiskTechnology) ?",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed="5")
A

In the SPARQL request provided, the PREFIX statement is used to define a namespace 'pbn' for the ontology. In this case, it is linked to the ontology in question. 
The SELECT statement is then used to count the number of distinct classes in the knowledge graph. The COUNT and DISTINCT functions are employed within the SELECT statement to achieve this. The COUNT function counts the number of instances found, while the DISTINCT function ensures that only unique instances are counted. The WHERE statement is utilized to specify the pattern to match, which in this case is any instance (?s) that belongs to a class (?class).
Once the request is executed against the knowledge graph, it will return the count of distinct classes present in the ontology.

--------

```sparql
PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT (COUNT(DISTINCT ?class) as ?classCount)
WHERE {
    ?s a ?class
}```

--------

[[38]]

In [14]:
A = ask(specifics+"How do I count the top 5 classes that have the children with most instances?",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed="")
A

In the given sparql query, we use the provided ontology prefix to specify the namespace of the classes and properties. The query starts by selecting the variables ?class and ?child. We then match all instances ?child of each class ?class using the triple pattern ?child a ?class. The COUNT function is used to count the number of instances for each class, and the results are grouped by ?class. The GROUP BY clause ensures that the count is performed for each unique class. We then order the results by the count in descending order using ORDER BY DESC(?count). Finally, we limit the results to the top 5 classes using the LIMIT 5 clause.

--------

```sparql
PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT ?class (COUNT(?child) AS ?count)
WHERE {
  ?child a ?class .
} 
GROUP BY ?class
ORDER BY DESC(?count)
LIMIT 5
```

--------

[[owl.NamedIndividual, 88171],
 [WIP.Technology, 23139],
 [WIP.Mitigation, 22950],
 [WIP.Risk, 21159],
 [WIP.Stakeholder, 16128]]

In [15]:
A = ask(specifics+"I have a specific Risk. How can I search for the list of Stakeholder impacted?",overwrite=False,MODEL="gpt-3.5-turbo-1106",seed="")

In the provided SPARQL request, we start by defining a prefix for the ontology namespace using the `PREFIX` keyword. We use `pbn` as the prefix for `https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#`.

We then use the `SELECT` clause to specify the variables we want to retrieve from the query. In this case, we want to count the number of distinct classes, so we use the `COUNT` function and alias the result as `?numberOfClasses`.

The `WHERE` clause is used to specify the patterns to match in the data. In this case, we want to find all resources that are instances of the `owl:Class` class. The `?class` variable is used as a placeholder for these instances.

Overall, the query will count the number of distinct classes in the knowledge graph by matching instances of the `owl:Class` class.

--------

```sparql
PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT (COUNT(DISTINCT ?class) as ?numberOfClasses)
WHERE {
  ?class a owl:Class .
}
```

--------

In [28]:
A = EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT DISTINCT ?tech ?risk ?riskSubgroup (COUNT(?riskGroup) AS ?techCount) ?techSubgroup
WHERE {
  ?techSubgroup a pbn:TechSubGroup .
  ?tech  pbn:has_TechSubgroup ?techSubgroup .
  ?risk pbn:has_RiskTechnology ?tech .
  ?risk pbn:has_RiskSubgroup ?riskSubgroup .
  FILTER (?tech = pbn:PBN__Technology_1010)  # Replace 'SpecificTechnology' with the actual technology of interest
}
GROUP BY ?riskGroup
ORDER BY DESC(?techCount)
LIMIT 5""")
A

[[WIP.PBN__Technology_1010,
  WIP.PBN__Risk_4243,
  WIP.PBN__RiskSubgroup_46,
  40,
  WIP.PBN__TechSubGroup_75]]

In [24]:
[[x[0].label[0],x[1].label[0],x[2].label[0],x[4].label[0]] for x in A]

[['Natural ventilation systems',
  'Inadequate natural ventilation in hospital and built environments',
  'Lack of emphasis on air quality',
  'Green infrastructure management'],
 ['Natural ventilation systems',
  'Inadequate natural ventilation in hospital and built environments',
  'Lack of emphasis on air quality',
  'Green infrastructure management'],
 ['Natural ventilation systems',
  'Inadequate natural ventilation in hospital and built environments',
  'Lack of emphasis on air quality',
  'Green infrastructure management'],
 ['Natural ventilation systems',
  'Inadequate natural ventilation in hospital and built environments',
  'Lack of emphasis on air quality',
  'Green infrastructure management'],
 ['Natural ventilation systems',
  'Inadequate natural ventilation in hospital and built environments',
  'Lack of emphasis on air quality',
  'Green infrastructure management']]

In [19]:
A = EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT ?tech ?risk ?riskSubgroup (COUNT(?riskGroup) AS ?techCount) ?techSubgroup
WHERE {
  ?tech a pbn:Technology .
  ?tech  pbn:has_TechSubgroup ?techSubgroup .
  ?risk pbn:has_RiskTechnology ?tech .
  ?risk pbn:has_RiskSubgroup ?riskSubgroup .
  FILTER (?tech = pbn:PBN__Technology_1010)  # Replace 'SpecificTechnology' with the actual technology of interest
}
GROUP BY ?riskGroup
ORDER BY DESC(?techCount)
LIMIT 5""")
A

[[WIP.PBN__Technology_1010,
  WIP.PBN__Risk_4243,
  WIP.PBN__RiskSubgroup_46,
  80,
  WIP.PBN__Technology_2117],
 [WIP.PBN__Technology_1010,
  WIP.PBN__Risk_4243,
  WIP.PBN__RiskSubgroup_46,
  80,
  WIP.PBN__Technology_2117],
 [WIP.PBN__Technology_1010,
  WIP.PBN__Risk_4243,
  WIP.PBN__RiskSubgroup_46,
  80,
  WIP.PBN__Technology_2117],
 [WIP.PBN__Technology_1010,
  WIP.PBN__Risk_4243,
  WIP.PBN__RiskSubgroup_46,
  80,
  WIP.PBN__Technology_2117],
 [WIP.PBN__Technology_1010,
  WIP.PBN__Risk_4243,
  WIP.PBN__RiskSubgroup_46,
  80,
  WIP.PBN__Technology_2117]]

In [41]:
onto.PBN__Technology_2117.is_a

[WIP.Technology]

In [32]:
[[x[0].label[0],x[1].label[0],x[2].label[0],x[3].label[0]] for x in A]

[['Hvac and ventilation systems',
  'Natural ventilation systems',
  'Inadequate natural ventilation in hospital and built environments',
  'Lack of emphasis on air quality'],
 ['Hvac and ventilation systems',
  'Natural ventilation systems',
  'Open windows may lead to the uneven distribution of pollutants in a building, with some areas experiencing rapid reduction in pollutant concentration while others take longer to clear.',
  'Air quality and pollution effects'],
 ['Hvac and ventilation systems',
  'Natural ventilation systems',
  'Decreased transmission during summer period by 22%',
  'Climate change effects'],
 ['Hvac and ventilation systems',
  'Natural ventilation systems',
  'Human fear of infectious diseases has led to the emergence of artificial environments.',
  'Health behaviors and compliance'],
 ['Hvac and ventilation systems',
  'Natural ventilation systems',
  'High scores for lighting and electricity domains, impacting energy efficiency and potential to reduce diseas

In [123]:
EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT DISTINCT ?riskGroup ?risk ?technology
WHERE {
  ?riskGroup a pbn:RiskGroup .
  ?risk pbn:has_RiskGroup ?riskGroup .
  ?risk pbn:has_RiskTechnology ?technology .
  FILTER (?riskGroup = pbn:PBN__RiskGroup_4)  # Replace 'SpecificRiskGroup' with the actual RiskGroup of interest
}
ORDER BY ?technology  # or another property if you have a ranking metric
""")

[[WIP.PBN__RiskGroup_4, WIP.PBN__Risk_2240, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_1463, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_1243, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_1163, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_1763, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_1627, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_356, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_2365, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_1144, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_2293, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_1649, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_1628, WIP.PBN__Technology_22],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_264, WIP.PBN__Technology_358],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Risk_939, WIP.PBN__Technology_185],
 [WIP.PBN__RiskGroup_4, WIP.PBN__Ri

In [98]:
EX("""FILTER""")

ParsingError: ('Error at FILTER', 0)

In [208]:
A = EX("""PREFIX pbn: <https://github.com/mm80843/T3.5/raw/main/pbn_t3_5.owl#>

SELECT ?riskGroup ?techSubgroup (COUNT(?techSubgroup) AS ?techCount)
WHERE {
  ?riskGroup a pbn:RiskGroup .
  ?risk pbn:has_RiskGroup ?riskGroup .
  ?risk pbn:has_RiskTechnology ?technology .
  ?technology a pbn:Technology . 
  ?riskGroup a pbn:RiskGroup . 
  ?technology pbn:has_TechSubgroup ?techSubgroup . 
}
GROUP BY ?techSubgroup
ORDER BY DESC(?techCount)
LIMIT 10""")
A

[[WIP.PBN__RiskGroup_6, WIP.PBN__TechGroup_11, 2205],
 [WIP.PBN__RiskGroup_6, WIP.PBN__TechGroup_54, 1993],
 [WIP.PBN__RiskGroup_6, WIP.PBN__TechGroup_25, 1846],
 [WIP.PBN__RiskGroup_6, WIP.PBN__TechGroup_16, 1639],
 [WIP.PBN__RiskGroup_6, WIP.PBN__Technology_22, 1622],
 [WIP.PBN__RiskGroup_6, WIP.PBN__TechGroup_20, 1534],
 [WIP.PBN__RiskGroup_6, WIP.PBN__TechGroup_10, 1435],
 [WIP.PBN__RiskGroup_6, WIP.PBN__TechGroup_39, 1316],
 [WIP.PBN__RiskGroup_6, WIP.PBN__Technology_2120, 1264],
 [WIP.PBN__RiskGroup_6, WIP.PBN__Technology_2119, 1163]]

In [203]:
onto.PBN__Technology_22.get_properties()

{WIP.has_TechGroup,
 rdf-schema.label,
 WIP.has_Closest,
 WIP.has_TechnologyRisk,
 WIP.has_Category,
 WIP.has_TechSubgroup,
 WIP.has_TechSource,
 WIP.has_TechnologyBlueprint,
 WIP.has_SubgroupTech,
 WIP.has_TechnologyMitigation}

In [180]:
onto.PBN__RiskGroup_6.label[0]

'Public health risks'

In [181]:
[x[1].label[0] for x in A]

['Urban planning',
 'Data analytics',
 'Ai tools',
 'Iot',
 'Digital and information management',
 'Infrastructure',
 'Sensors and monitoring',
 'Monitoring']

In [67]:
for x in onto.RiskGroup.instances():
    print(x)

WIP.PBN__RiskGroup_6
WIP.PBN__RiskGroup_29
WIP.PBN__RiskGroup_49
WIP.PBN__RiskGroup_7
WIP.PBN__RiskGroup_57
WIP.PBN__RiskGroup_18
WIP.PBN__RiskGroup_36
WIP.PBN__RiskGroup_45
WIP.PBN__RiskGroup_20
WIP.PBN__RiskGroup_70
WIP.PBN__RiskGroup_2
WIP.PBN__RiskGroup_66
WIP.PBN__RiskGroup_3
WIP.PBN__RiskGroup_78
WIP.PBN__RiskGroup_4
WIP.PBN__RiskGroup_76
WIP.PBN__RiskGroup_13
WIP.PBN__RiskGroup_32
WIP.PBN__RiskGroup_8
WIP.PBN__RiskGroup_41
WIP.PBN__RiskGroup_30
WIP.PBN__RiskGroup_10
WIP.PBN__RiskGroup_31
WIP.PBN__RiskGroup_88
WIP.PBN__RiskGroup_83
WIP.PBN__RiskGroup_0
WIP.PBN__RiskGroup_15
WIP.PBN__RiskGroup_87
WIP.PBN__RiskGroup_19
WIP.PBN__RiskGroup_22
WIP.PBN__RiskGroup_23
WIP.PBN__RiskGroup_79
WIP.PBN__RiskGroup_65
WIP.PBN__RiskGroup_43
WIP.PBN__RiskGroup_54
WIP.PBN__RiskGroup_84
WIP.PBN__RiskGroup_64
WIP.PBN__RiskGroup_1
WIP.PBN__RiskGroup_28
WIP.PBN__RiskGroup_33
WIP.PBN__RiskGroup_16
WIP.PBN__RiskGroup_21
WIP.PBN__RiskGroup_62
WIP.PBN__RiskGroup_56
WIP.PBN__RiskGroup_82
WIP.PBN__RiskGroup